In [17]:
import random
import copy

In [18]:
graph = [[0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [3, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 7, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
                           [0, 0, 9, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [2, 0, 0, 0, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0],
                           [0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 8, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 9, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 1],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [19]:
# matrica tuple-ova: vrsta oznacava polazni cvor, prvi element krajnji, drugi tezinu, treci feromone
def make_pheromone_graph(graph):
    pheromone_graph = []
    for i in range(len(graph)):
        neighbours = []
        for j in range(len(graph[0])):
            if graph[i][j] == 0:
                continue
            neighbour = [j, graph[i][j], 0.1]
            neighbours.append(neighbour)
        pheromone_graph.append(neighbours)
    return pheromone_graph

In [20]:
pheromone_graph = make_pheromone_graph(graph)
copy_of_pheromone_graph = copy.deepcopy(pheromone_graph)
print(pheromone_graph)

[[[1, 3, 0.1], [5, 2, 0.1]], [[0, 3, 0.1], [3, 7, 0.1], [7, 1, 0.1]], [[4, 9, 0.1]], [[1, 7, 0.1], [9, 5, 0.1]], [[2, 9, 0.1], [5, 4, 0.1]], [[0, 2, 0.1], [4, 4, 0.1], [6, 6, 0.1]], [[5, 6, 0.1]], [[1, 1, 0.1], [8, 2, 0.1]], [[7, 2, 0.1], [9, 1, 0.1]], [[3, 5, 0.1], [8, 1, 0.1], [10, 4, 0.1]], [[9, 4, 0.1], [11, 6, 0.1]], [[10, 6, 0.1], [12, 8, 0.1]], [[11, 8, 0.1], [13, 9, 0.1]], [[12, 9, 0.1], [14, 1, 0.1]], [[13, 1, 0.1]]]


In [21]:
# dodajemo sve grane koje vode ka neposecenim cvorovima
def add_neighbours(visited, neighbours, pheromone_graph):
    for node in visited:
        for branch in pheromone_graph[node]:
            if branch[0] not in visited:
                neighbours.append(branch)

In [22]:
def traverse(i, pheromone_graph, alpha, beta, k):
    n = len(pheromone_graph)
    tree = []
    tree_weight = 0
    visited = [i]
    while len(visited) < k:
        neighbours = []
        add_neighbours(visited, neighbours, pheromone_graph)
        if len(neighbours) == 0:
            break
        values = [v[2]**alpha / v[2]**beta for v in neighbours]
        chosen_neighbour = random.choices(neighbours, weights = values, k = 1)[0]
        tree.append((i, chosen_neighbour))
        tree_weight += chosen_neighbour[1]
        visited.append(chosen_neighbour[0])
        
    
    if tree_weight != float('inf') and len(visited) == k:
        return tree, tree_weight

In [23]:
traverse(0, pheromone_graph, 0.1, 0.1, 3)

([(0, [1, 3, 0.1]), (0, [5, 2, 0.1])], 5)

In [24]:
def aco(pheromone_graph, num_ants, rho, num_iters, theta, alpha, beta, k):
    n = len(pheromone_graph)
    best_tree = None
    
    for j in range(num_iters):
        pheromone_graph = copy.deepcopy(copy_of_pheromone_graph)
        node_best_tree = None
        
        for i in range(n):
            trees = [traverse(i, pheromone_graph, alpha, beta, k) for _ in range(num_ants)]

            check = None
            for tree in trees:
                if tree is not None:
                    check = copy.deepcopy(tree)

            if check is None:
                break

            for u in range(n):
                for v in range(u + 1, n):
                    if v not in pheromone_graph[u]:
                        continue
                    pheromone_graph[u][v]['pheromones'] *= rho

            for tree, tree_weight in trees:
                delta = theta / tree_weight
                for branch in tree:
                    branch[1][2] += delta

            ants_best_tree = min(trees, key = lambda c: c[1])
            if node_best_tree is None or ants_best_tree[1] < node_best_tree[1]:
                node_best_tree = copy.deepcopy(ants_best_tree)

        if best_tree is None or node_best_tree[1] < best_tree[1]:
                best_tree = copy.deepcopy(node_best_tree)

    if best_tree is not None:
        print(best_tree)
    else:
        print("No such tree")

In [25]:
import time
start = time.time()
aco(pheromone_graph, 10, 0.9, 100, 0.9, 1.5, 100.0, 10)
end = time.time()
print(end - start)

([(8, [7, 2, 1.2483917433917444]), (8, [1, 1, 0.7005357142857144]), (8, [0, 3, 1.1723216441361601]), (8, [5, 2, 1.2742780415860262]), (8, [6, 6, 1.2737496042430734]), (8, [9, 1, 1.3996891415036579]), (8, [3, 5, 1.0774813492958653]), (8, [4, 4, 1.4345764931344775]), (8, [10, 4, 1.4885159089817883])], 28)
2.6776187419891357


In [27]:
%store -r random_graph
pheromone_graph = make_pheromone_graph(random_graph)
copy_of_pheromone_graph = copy.deepcopy(pheromone_graph)
start = time.time()
aco(pheromone_graph, 1, 0.9, 1, 0.9, 1.5, 100.0, 10)
end = time.time()
print(end - start)

KeyboardInterrupt: 